In [1]:
import json
import os
import argparse

import numpy as np
import random
import torch
from torch import nn, optim
from tqdm.auto import tqdm

from pathlib import Path
from torch.utils.data import DataLoader, Subset
import sys
sys.path.append('../../lib/exlib/src')
from exlib.modules.sop import SOPImageCls, SOPConfig, get_chained_attr
from exlib.datasets.cosmogrid import CosmogridDataset, CNNModel

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

SEED = 42
if SEED != -1:
    # Torch RNG
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    # Python RNG
    np.random.seed(SEED)
    random.seed(SEED)

In [3]:
# model paths
backbone_model_name = '../../data/cosmogrid/CNN_mass_maps.pth'

# data paths
TRAIN_DATA_DIR = '../../data/cosmogrid'
VAL_DATA_DIR = '../../data/cosmogrid'
mask_path = '../../data/cosmogrid/masks/X_maps_Cosmogrid_100k_watershed_diagonal.npy'

# training args
batch_size = 16
lr = 0.0005
num_epochs = 20
warmup_steps = 2000
mask_batch_size = 64

# experiment args
exp_dir = '../../exps/cosmogrid_4h_nmm100_bf_even_ep20_pis32_gg1.0_gs1.0_ggta1.0_ggtb0.1_bblNone_ihs32_segseg/best'
# exp_dir = '../../exps/cosmogrid_4h_nmm100_bf_even_ep4_pis32_gg1.0_gs1.0_ggta1.0_ggtb0.1_bblNone_ihs32_segseg/best'
# exp_dir = '../../exps/cosmogrid_4h_nmm100_bf_even_ep4_pis16_gg1.0_gs1.0_ggta1.0_ggtb0.1_bblNone_ihs32_segseg/best'
# exp_dir = '../../exps/cosmogrid_1h_nmm100_bf_even_ep1_pis16_gg1.0_gs1.0_ggta1.0_ggtb0.1_bblNone_ihs32_segseg/best'
# exp_dir = '../../exps/cosmogrid_4h_nmm100_bf_even_ep20_pis16_gg1.0_gs1.0/best'
os.makedirs(exp_dir, exist_ok=True)
output_dirname = os.path.join(exp_dir, 'val_results')
os.makedirs(output_dirname, exist_ok=True)

In [4]:
config = SOPConfig(os.path.join(exp_dir, 'config.json'))

backbone_model = CNNModel(config.num_labels)
state_dict = torch.load(backbone_model_name)
backbone_model.load_state_dict(state_dict=state_dict)
processor = None

In [5]:
def mask_transform(mask):
    seg_mask_cut_off = config.num_masks_max
    # Preprocess the mask using the ViTImageProcessor
    if len(mask.shape) == 2 and mask.dtype == torch.bool:
        mask_dim1, mask_dim2 = mask.shape
        mask = mask.unsqueeze(0).expand(3, 
                                        mask_dim1, 
                                        mask_dim2).float()
        if processor is not None:
            inputs = processor(mask, 
                            do_rescale=False, 
                            do_normalize=False,
                            return_tensors='pt')
            # (1, 3, 224, 224)
            return inputs['pixel_values'][0][0]
        else:
            return mask
    else: # len(mask.shape) == 3
        if mask.dtype != torch.bool:
            if len(mask.shape) == 2:
                mask = mask.unsqueeze(0)
            mask = convert_idx_masks_to_bool(mask)
        bsz, mask_dim1, mask_dim2 = mask.shape
        mask = mask.unsqueeze(1).expand(bsz, 
                                        3, 
                                        mask_dim1, 
                                        mask_dim2).float()

        if bsz < seg_mask_cut_off:
            repeat_count = seg_mask_cut_off // bsz + 1
            mask = torch.cat([mask] * repeat_count, dim=0)

        # add additional mask afterwards
        mask_sum = torch.sum(mask[:seg_mask_cut_off - 1], dim=0, keepdim=True).bool()
        if False in mask_sum:
            mask = mask[:seg_mask_cut_off - 1]
            compensation_mask = (1 - mask_sum.int()).bool()
            mask = torch.cat([mask, compensation_mask])
        else:
            mask = mask[:seg_mask_cut_off]

        if processor is not None:
            inputs = processor(mask, 
                            do_rescale=False, 
                            do_normalize=False,
                            return_tensors='pt')
            
            return inputs['pixel_values'][:,0]
        else:
            return mask[:,0]
        
def convert_idx_masks_to_bool(masks):
    """
    input: masks (1, img_dim1, img_dim2)
    output: masks_bool (num_masks, img_dim1, img_dim2)
    """
    unique_idxs = torch.sort(torch.unique(masks)).values
    idxs = unique_idxs.view(-1, 1, 1)
    broadcasted_masks = masks.expand(unique_idxs.shape[0], 
                                     masks.shape[1], 
                                     masks.shape[2])
    masks_bool = (broadcasted_masks == idxs)
    return masks_bool

In [6]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader


# Load the dataset
train_size, val_size = -1, -1
# train_size = 100
# val_size = 100
# train_dataset = CosmogridDataset(data_dir=TRAIN_DATA_DIR, split='train', data_size=train_size,
#                                  mask_path=mask_path, mask_transform=mask_transform)
val_dataset = CosmogridDataset(data_dir=TRAIN_DATA_DIR, split='val', data_size=train_size,
                                 mask_path=mask_path, mask_transform=mask_transform)

# Create a DataLoader to batch and shuffle the data
# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# samples used for training: 80000
# samples used for validation: 10000
# samples used for testing: 10000
# total samples: 100000
x shape (80000, 66, 66) (10000, 66, 66) (10000, 66, 66)
y shape (80000, 6) (10000, 6) (10000, 6)
masks shape (80000, 66, 66) (10000, 66, 66) (10000, 66, 66)
-- ALL --
max 0.6323861033355062
min -0.031224769235240986
-- SPLIT val --
max 0.6323861033355062
min -0.031224769235240986
Finished loading 10000 val images ... 


In [7]:
backbone_model = backbone_model.to(device)
class_weights = get_chained_attr(backbone_model, config.finetune_layers[0]).weight #.clone().to(device)
class_weights.shape
model = SOPImageCls(config, backbone_model) #, class_weights=class_weights, projection_layer=None)
model_state_dict = torch.load(os.path.join(exp_dir, 'checkpoint.pth'), map_location=device)['model']
model.load_state_dict(model_state_dict)
model = model.to(device)
model.eval();

projection layer is not frozen


In [9]:
# time 

import time

from tqdm.auto import tqdm

criterion = nn.MSELoss(reduction='none')


count = 0
start = time.time()
for batch in tqdm(val_dataloader):

    inputs, labels, masks, masks_i = batch
    inputs = inputs.to(device, dtype=torch.float)
    labels = labels.to(device, dtype=torch.float)
    masks = masks.to(device)

    
    with torch.no_grad():
        outputs = model(inputs, segs=masks, mask_batch_size=mask_batch_size, return_tuple=True, binary_threshold=0.5)
        
    count += 1
    if count == 10:
        break
print(time.time() - start)

  0%|          | 0/625 [00:00<?, ?it/s]

40.3492546081543


In [25]:
from tqdm.auto import tqdm

criterion = nn.MSELoss(reduction='none')


count = 0

for batch in tqdm(val_dataloader):

    inputs, labels, masks, masks_i = batch
    inputs = inputs.to(device, dtype=torch.float)
    labels = labels.to(device, dtype=torch.float)
    masks = masks.to(device)

    
    with torch.no_grad():
        outputs = model(inputs, segs=masks, mask_batch_size=mask_batch_size, return_tuple=True, binary_threshold=0.5)
        original_logits = backbone_model(inputs).logits

        logits = outputs.logits
        logits_all = outputs.logits_all
        masks = outputs.masks
        mask_weights = outputs.mask_weights
        attributions = outputs.attributions

        bsz, num_masks, num_labels = logits_all.shape

        loss_sop = criterion(logits, labels)
        loss_original = criterion(original_logits, labels)
        losses = criterion(logits_all.reshape(-1, model.config.num_labels), 
                         labels.unsqueeze(1).expand(bsz, 
                                                    num_masks, 
                                                    num_labels).reshape(-1, num_labels))
        losses = losses.reshape(logits_all.shape)

        for i in tqdm(range(bsz)):
            output_filepath = os.path.join(output_dirname, f'{count}.pt')
            # if os.path.exists(output_filepath):
            #     count += 1
            #     continue
            masks_used = masks[i][mask_weights[i].sum(-1).bool()]  # get masks that are used for any class
            mask_weights_used = mask_weights[i][mask_weights[i].sum(-1).bool()]
            logits_used = logits_all[i][mask_weights[i].sum(-1).bool()]
            losses_used = losses[i][mask_weights[i].sum(-1).bool()]

            unique_masks_used, reverse_indices, counts = torch.unique(masks_used, 
                                                                    dim=0, 
                                                                    return_inverse=True, 
                                                                    return_counts=True)
            indices = []
            reverse_indices = reverse_indices.cpu().numpy().tolist()

            for j in range(len(counts)):
                indices.append(reverse_indices.index(j))

            unique_mask_weights_used = mask_weights_used[indices] * counts.view(-1, 1)
            unique_logits_used = logits_used[indices]
            unique_losses_used = losses_used[indices]

            entry = {'image': inputs[i],
                     'original_logits': original_logits[i],
                    'logits': logits[i],
                    'unique_logits_used': unique_logits_used,
                    'masks': masks[i],
                    'unique_masks_used': unique_masks_used,
                    'unique_mask_weights_used': unique_mask_weights_used,
                    'unique_losses_used': unique_losses_used,
                    'label': labels[i],
                    'counts': counts,
                    'num_labels': model.config.num_labels}

            torch.save(entry, output_filepath)
            count += 1

  0%|          | 0/625 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

...

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

In [ ]:
output_filepath

In [ ]:
from tqdm.auto import tqdm

criterion = nn.MSELoss(reduction='none')


count = 0

for batch in tqdm(val_dataloader):

    inputs, labels, masks, masks_i = batch
    inputs = inputs.to(device, dtype=torch.float)
    labels = labels.to(device, dtype=torch.float)
    masks = masks.to(device)

    
    with torch.no_grad():
        outputs = model(inputs, segs=masks, mask_batch_size=mask_batch_size, return_tuple=True, binary_threshold=0.5)
        original_logits = backbone_model(inputs).logits

        logits = outputs.logits
        logits_all = outputs.logits_all
        masks = outputs.masks
        mask_weights = outputs.mask_weights
        attributions = outputs.attributions

        bsz, num_masks, num_labels = logits_all.shape

        loss_sop = criterion(logits, labels)
        loss_original = criterion(original_logits, labels)
        losses = criterion(logits_all.reshape(-1, model.config.num_labels), 
                         labels.unsqueeze(1).expand(bsz, 
                                                    num_masks, 
                                                    num_labels).reshape(-1, num_labels))
        losses = losses.reshape(logits_all.shape)

        for i in tqdm(range(bsz)):
            output_filepath = os.path.join(output_dirname, f'{count}.pt')
            # if os.path.exists(output_filepath):
            #     count += 1
            #     continue
            masks_used = masks[i][mask_weights[i].sum(-1).bool()]  # get masks that are used for any class
            mask_weights_used = mask_weights[i][mask_weights[i].sum(-1).bool()]
            logits_used = logits_all[i][mask_weights[i].sum(-1).bool()]
            losses_used = losses[i][mask_weights[i].sum(-1).bool()]

            unique_masks_used, reverse_indices, counts = torch.unique(masks_used, 
                                                                    dim=0, 
                                                                    return_inverse=True, 
                                                                    return_counts=True)
            indices = []
            reverse_indices = reverse_indices.cpu().numpy().tolist()

            for j in range(len(counts)):
                indices.append(reverse_indices.index(j))

            unique_mask_weights_used = mask_weights_used[indices] * counts.view(-1, 1)
            unique_logits_used = logits_used[indices]
            unique_losses_used = losses_used[indices]

            print(masks[i])
            entry = {'image': inputs[i],
                     'original_logits': original_logits[i],
                    'logits': logits[i],
                    'unique_logits_used': unique_logits_used,
                    'masks': masks[i],
                    'unique_masks_used': unique_masks_used,
                    'unique_mask_weights_used': unique_mask_weights_used,
                    'unique_losses_used': unique_losses_used,
                    'label': labels[i],
                    'counts': counts,
                    'num_labels': model.config.num_labels}
            
            break

            torch.save(entry, output_filepath)
            count += 1
        break

In [12]:
entry['mask']

KeyError: 'mask'

In [10]:
model.training

False

In [9]:
import matplotlib.pyplot as plt
import matplotlib

unique_bool_masks, summed_weights, mean_loss = sum_weights_for_unique_masks(entry['unique_masks_used'], 
                                                                       entry["unique_mask_weights_used"], 
                                                                       entry["unique_losses_used"])

for j in range(len(unique_bool_masks)):
    weight_show = [round(val.item(), 4) for val in summed_weights[j]]
    loss_show = [round(val.item(), 4) for val in mean_loss[j]]
    label_show = [round(val.item(), 4) for val in entry["label"]]
    title = f'Weight: {weight_show}\nLoss: {loss_show}\n{label_show}'
    map_plotter(entry['image'][0].cpu(), unique_bool_masks[j].cpu().numpy(), type='dim')
    plt.title(title)
    plt.axis('off')
    plt.show()

NameError: name 'sum_weights_for_unique_masks' is not defined

In [29]:
indices

[0]

In [30]:

outputs_avg, outputs, attn_weights1, attn_weights2, pooler_outputs = model(inputs, 
                                                                           segs=masks,
                                                                           mask_batch_size=mask_batch_size, 
                                                                           return_tuple=True)
original_outputs = backbone_model(inputs).logits

preds_wrapped.append(outputs_avg)
preds_original.append(original_outputs)
gold_labels.append(labels)

bsz, num_masks, num_labels = outputs.shape

loss_avg = criterion(outputs_avg, labels)
total_loss_wrapped += loss_avg.sum(0)
loss_original = criterion(original_outputs, labels)
total_loss_original += loss_original.sum(0)
total_count += labels.size(0)
loss = criterion(outputs.reshape(-1, model.config.num_labels), 
                 labels.unsqueeze(1).expand(bsz, 
                                            num_masks, 
                                            num_labels).reshape(-1, num_labels))
loss = loss.reshape(outputs.shape)

i = 0
masks_used = attn_weights1[i][attn_weights2[i].sum(-1).bool()]  # get masks that are used for any class
mask_weights = attn_weights2[i][attn_weights2[i].sum(-1).bool()]
output_used = outputs[i][attn_weights2[i].sum(-1).bool()]
pooler_used = pooler_outputs[i][attn_weights2[i].sum(-1).bool()]
loss_used = loss[i][attn_weights2[i].sum(-1).bool()]

# import pdb
# pdb.set_trace()
unique_masks_used, reverse_indices, counts = torch.unique(masks_used, 
                                                        dim=0, 
                                                        return_inverse=True, 
                                                        return_counts=True)
indices = []
reverse_indices = reverse_indices.cpu().numpy().tolist()

for j in range(len(counts)):
    indices.append(reverse_indices.index(j))
# import pdb
# pdb.set_trace()
unique_mask_weights = mask_weights[indices] * counts.view(-1, 1)
unique_outputs = output_used[indices]
unique_preds = loss_used[indices].tolist()
unique_pooler = pooler_used[indices]

entry = {'image': images[i],
        'outputs_avg': outputs_avg[i],
        'outputs': unique_outputs, # outputs[i]
        'pooler': unique_pooler,
        'outputs_original': original_outputs[i],
        'masks': attn_weights1[i].cpu().numpy(),
        'masks_all': masks_i[i].cpu().numpy() \
            if masks_i is not None \
                else None,
        'masks_used': unique_masks_used,  # masks_used,
        'mask_weights': unique_mask_weights,  # mask_weights,
        'pred': loss_avg[i].tolist(),
        'preds': unique_preds, # predicted_raw[i].tolist(),
        'label': labels[i].tolist(),
        'counts': counts.cpu().numpy(),
        'num_labels': model.config.num_labels}


OutOfMemoryError: CUDA out of memory. Tried to allocate 664.67 GiB (GPU 0; 47.54 GiB total capacity; 1.68 GiB already allocated; 4.89 GiB free; 42.32 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
criterion = nn.MSELoss(reduction='none')

all_results = []

progress_bar = tqdm(range(len(val_dataloader)))
model.eval()
preds_wrapped = []
preds_original = []
gold_labels = []
with torch.no_grad():
    idx = 0
    total_loss_wrapped = 0
    total_loss_original = 0
    total_count = 0
    for batch in val_loader:
        output_filename = os.path.join(output_dirname, f'{idx}.pt')
        if os.path.exists(output_filename):
            progress_bar.update(1)
            idx += 1
            continue
        inputs, labels, masks, masks_i = batch
        inputs = inputs.to(device, dtype=torch.float)
        labels = labels.to(device, dtype=torch.float)
        masks = masks.to(device)

        images = inputs.cpu().numpy()

        # images = [Image.fromarray(inputs_numpy[i]) for i in range(inputs_numpy.shape[0])]

        outputs_avg, outputs, attn_weights1, attn_weights2, pooler_outputs = model(inputs, 
                                                                   masks=masks,
                                epoch=model.config.num_heads,
                                mask_batch_size=args.mask_batch_size,
                                return_pooler=True)
        original_outputs = backbone_model(inputs).logits

        preds_wrapped.append(outputs_avg)
        preds_original.append(original_outputs)
        gold_labels.append(labels)

        bsz, num_masks, num_labels = outputs.shape

        loss_avg = criterion(outputs_avg, labels)
        total_loss_wrapped += loss_avg.sum(0)
        loss_original = criterion(original_outputs, labels)
        total_loss_original += loss_original.sum(0)
        total_count += labels.size(0)
        loss = criterion(outputs.reshape(-1, model.config.num_labels), 
                         labels.unsqueeze(1).expand(bsz, 
                                                    num_masks, 
                                                    num_labels).reshape(-1, num_labels))
        loss = loss.reshape(outputs.shape)

        if not args.scatter_only:
            for i in range(len(images)):

                masks_used = attn_weights1[i][attn_weights2[i].sum(-1).bool()]  # get masks that are used for any class
                mask_weights = attn_weights2[i][attn_weights2[i].sum(-1).bool()]
                output_used = outputs[i][attn_weights2[i].sum(-1).bool()]
                pooler_used = pooler_outputs[i][attn_weights2[i].sum(-1).bool()]
                loss_used = loss[i][attn_weights2[i].sum(-1).bool()]

                # import pdb
                # pdb.set_trace()
                unique_masks_used, reverse_indices, counts = torch.unique(masks_used, 
                                                                        dim=0, 
                                                                        return_inverse=True, 
                                                                        return_counts=True)
                indices = []
                reverse_indices = reverse_indices.cpu().numpy().tolist()

                for j in range(len(counts)):
                    indices.append(reverse_indices.index(j))
                # import pdb
                # pdb.set_trace()
                unique_mask_weights = mask_weights[indices] * counts.view(-1, 1)
                unique_outputs = output_used[indices]
                unique_preds = loss_used[indices].tolist()
                unique_pooler = pooler_used[indices]

                entry = {'image': images[i],
                        'outputs_avg': outputs_avg[i],
                        'outputs': unique_outputs, # outputs[i]
                        'pooler': unique_pooler,
                        'outputs_original': original_outputs[i],
                        'masks': attn_weights1[i].cpu().numpy(),
                        'masks_all': masks_i[i].cpu().numpy() \
                            if masks_i is not None \
                                else None,
                        'masks_used': unique_masks_used,  # masks_used,
                        'mask_weights': unique_mask_weights,  # mask_weights,
                        'pred': loss_avg[i].tolist(),
                        'preds': unique_preds, # predicted_raw[i].tolist(),
                        'label': labels[i].tolist(),
                        'counts': counts.cpu().numpy(),
                        'num_labels': model.config.num_labels}

                # print('attn_weights1[i]', attn_weights1[i].shape)
                # print('attn_weights2[i]', attn_weights2[i].shape)

                all_results.append(entry)
                output_filename = os.path.join(output_dirname, f'{idx}.pkl')
                torch.save(entry, output_filename)

        progress_bar.update(1)
        idx += 1

        del images

    preds_wrapped = torch.cat(preds_wrapped).cpu().numpy()
    preds_original = torch.cat(preds_original).cpu().numpy()
    gold_labels = torch.cat(gold_labels).cpu().numpy()

    plot_y = gold_labels
    # predictions = preds
    upp_lims = np.nanmax(plot_y, axis=0)
    low_lims = np.nanmin(plot_y, axis=0)

    # Visualize predictions
    fig, axes = plt.subplots(nrows=1, ncols=len(low_lims), figsize=(20, 10))

    for ind, (low_lim, upp_lim) in enumerate(zip(low_lims, upp_lims)):
        p_w = np.poly1d(np.polyfit(plot_y[:, ind], preds_wrapped[:, ind], 1))
        p_o = np.poly1d(np.polyfit(plot_y[:, ind], preds_original[:, ind], 1))

        axes[ind].scatter(plot_y[:, ind], preds_wrapped[:, ind], color="blue", label='Wrapped')
        axes[ind].scatter(plot_y[:, ind], preds_original[:, ind], color="orange", label='Original')
        axes[ind].plot([low_lim, upp_lim], [low_lim, upp_lim], color="black")
        axes[ind].plot([low_lim, upp_lim], [p_w(low_lim), p_w(upp_lim)], color="black", ls=":", label='Wrapped')
        axes[ind].plot([low_lim, upp_lim], [p_o(low_lim), p_o(upp_lim)], color="black", ls="-.", label='Original')
        axes[ind].set_xlim([low_lim, upp_lim])
        axes[ind].set_ylim([low_lim, upp_lim])
        axes[ind].set_xlabel('Gold')
        axes[ind].set_ylabel('Pred')
        axes[ind].legend()
        axes[ind].set_aspect('equal', adjustable='box')

        original_loss = round((total_loss_original / total_count)[ind].cpu().item(), 4)
        wrapped_loss = round((total_loss_wrapped / total_count)[ind].cpu().item(), 4)

        axes[ind].set_title(f'Output {ind}, \n' + 
                            f'Original loss {original_loss},\n' +
                            f'Wrapped loss {wrapped_loss}')

    # plt.tight_layout()
    plt.subplots_adjust(left=0.05, right=0.95, bottom=0.05, top=0.9)
    plt.savefig(os.path.join(args.exp_dir, 'combined_scatter.png'))
    plt.close()

    # Visualize predictions
    fig, axes = plt.subplots(nrows=1, ncols=len(low_lims), figsize=(20, 10))

    for ind, (low_lim, upp_lim) in enumerate(zip(low_lims, upp_lims)):
        p_w = np.poly1d(np.polyfit(plot_y[:, ind], preds_wrapped[:, ind], 1))
        p_o = np.poly1d(np.polyfit(plot_y[:, ind], preds_original[:, ind], 1))

        # (Wrapped - Gold) / Gold
        diff_wrapped = (preds_wrapped[:, ind] - plot_y[:, ind]) / plot_y[:, ind]
        # (Original - Gold) / Gold
        diff_original = (preds_original[:, ind] - plot_y[:, ind]) / plot_y[:, ind]
        axes[ind].scatter(plot_y[:, ind], diff_wrapped, color="green", label='Wrapped')
        axes[ind].scatter(plot_y[:, ind], diff_original, color="red", label='Original')
        axes[ind].scatter(plot_y[:, ind], preds_wrapped[:, ind], color="blue", label='Wrapped')
        axes[ind].scatter(plot_y[:, ind], preds_original[:, ind], color="orange", label='Original')
        axes[ind].plot([low_lim, upp_lim], [low_lim, upp_lim], color="black")
        axes[ind].plot([low_lim, upp_lim], [p_w(low_lim), p_w(upp_lim)], color="black", ls=":", label='Wrapped')
        axes[ind].plot([low_lim, upp_lim], [p_o(low_lim), p_o(upp_lim)], color="black", ls="-.", label='Original')
        # axes[ind].set_xlim([low_lim, upp_lim])
        # axes[ind].set_ylim([low_lim, upp_lim])
        axes[ind].set_xlabel('Gold')
        axes[ind].set_ylabel('Pred')
        axes[ind].legend()
        axes[ind].set_aspect('equal', adjustable='box')

        original_loss = round((total_loss_original / total_count)[ind].cpu().item(), 4)
        wrapped_loss = round((total_loss_wrapped / total_count)[ind].cpu().item(), 4)

        axes[ind].set_title(f'Output {ind}, \n' + 
                            f'Original loss {original_loss},\n' +
                            f'Wrapped loss {wrapped_loss}')

    # plt.tight_layout()
    plt.subplots_adjust(left=0.05, right=0.95, bottom=0.05, top=0.9)
    plt.savefig(os.path.join(args.exp_dir, 'combined_scatter_diff.png'))
    plt.close()

    # Visualize predictions
    fig, axes = plt.subplots(nrows=1, ncols=len(low_lims), figsize=(20, 10))


    for ind, (low_lim, upp_lim) in enumerate(zip(low_lims, upp_lims)):
        p_w = np.poly1d(np.polyfit(plot_y[:, ind], preds_wrapped[:, ind], 1))
        p_o = np.poly1d(np.polyfit(plot_y[:, ind], preds_original[:, ind], 1))

        # (Wrapped - Gold) / Gold
        diff_wrapped = (preds_wrapped[:, ind] - plot_y[:, ind]) / plot_y[:, ind]
        # (Original - Gold) / Gold
        diff_original = (preds_original[:, ind] - plot_y[:, ind]) / plot_y[:, ind]
        axes[ind].scatter(plot_y[:, ind], diff_wrapped, color="blue", label='Wrapped')
        axes[ind].scatter(plot_y[:, ind], diff_original, color="orange", label='Original')
        # axes[ind].plot([low_lim, upp_lim], [low_lim, upp_lim], color="black")
        # axes[ind].plot([low_lim, upp_lim], [p_w(low_lim), p_w(upp_lim)], color="black", ls=":", label='Wrapped')
        # axes[ind].plot([low_lim, upp_lim], [p_o(low_lim), p_o(upp_lim)], color="black", ls="-.", label='Original')
        # axes[ind].set_xlim([low_lim, upp_lim])
        # axes[ind].set_ylim([low_lim, upp_lim])
        axes[ind].set_xlabel('Gold')
        axes[ind].set_ylabel('Pred')
        axes[ind].legend()
        axes[ind].set_aspect('equal', adjustable='box')

        original_loss = round((total_loss_original / total_count)[ind].cpu().item(), 4)
        wrapped_loss = round((total_loss_wrapped / total_count)[ind].cpu().item(), 4)

        axes[ind].set_title(f'Output {ind}, \n' + 
                            f'Original loss {original_loss},\n' +
                            f'Wrapped loss {wrapped_loss}')

    # plt.tight_layout()
    plt.subplots_adjust(left=0.05, right=0.95, bottom=0.05, top=0.9)
    plt.savefig(os.path.join(args.exp_dir, 'combined_scatter_diff_only.png'))
    plt.close()

    print('mean-center-offset', args.mean_center_offset)
    print('mean-center-scale', args.mean_center_scale)
    print('mean-center-offset2', args.mean_center_offset2)
    print('mean-center-scale2', args.mean_center_scale2)
    print('original_loss', original_loss)
    print('wrapped_loss', wrapped_loss)